In [1]:
#| default_exp protein_intensity_estimation

In [2]:
%reload_ext autoreload

%autoreload 2

# Protein Intenstiity Estimation

This notebook tests the protein LFQ intensity estimation step. It is in principle a wrapper around the functionality of the normalization.py class, which is used to shift precursors or transition intensity traces on top of each other.

# Unit Tests

#### Classes for testcase generation

In [3]:
import pandas as pd
import numpy as np
import directlfq.protein_intensity_estimation as lfq_protint
#test df cutting

def test_sorting_by_num_nans():
    vals1 = np.array([9, np.nan, np.nan, np.nan])
    vals2 = np.array([5, 6, np.nan, np.nan])
    vals3 = np.array([1, 2, 3,np.nan ])

    df = pd.DataFrame([vals1, vals2, vals3],index=[['P', 'P', 'P'],['A', 'B', 'C']])
    pcutter = lfq_protint.ProtvalCutter(df,maximum_df_length=2)
    sorted_idx = pcutter._sorted_idx
    df_sorted = df.loc[sorted_idx]
    
    assert np.allclose(df_sorted.iloc[2].to_numpy(), vals1,equal_nan=True)
    assert np.allclose(df_sorted.iloc[0].to_numpy(), vals3,equal_nan=True)
    

def test_cutting_of_df():
    vals1 = np.array([9, np.nan, np.nan, np.nan])
    vals2 = np.array([5, 6, np.nan, np.nan])
    vals3 = np.array([1, 2, 3,np.nan ])

    df = pd.DataFrame([vals1, vals2, vals3],index=[['A', 'B', 'C']])
    pcutter = lfq_protint.ProtvalCutter(df, maximum_df_length=2)
    cut_df = pcutter.get_dataframe()
    ion_idx = [x[0] for x in cut_df.index]
    print(ion_idx)
    assert ion_idx == ['C', 'B']


test_sorting_by_num_nans()
test_cutting_of_df()


['C', 'B']


In [4]:
import directlfq.normalization as lfq_norm
import directlfq.test_utils as lfq_testutils

def test_that_profiles_without_noise_are_shifted_exactly_on_top_of_each_other():
    peptide1= lfq_testutils.PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=3000, add_noise=False)
    peptide2= lfq_testutils.PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0.9, systematic_peptide_shift=3, add_noise=False)
    peptide3= lfq_testutils.PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=0.1, add_noise=False)
    peptide4= lfq_testutils.PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0.9, systematic_peptide_shift=100, add_noise=False)
    protein_df = lfq_testutils.ProteinProfileGenerator([peptide1, peptide2, peptide3, peptide4]).protein_profile_dataframe
    display(protein_df)
    normed_ion_profile = lfq_norm.normalize_ion_profiles(protein_df)
    display(normed_ion_profile)
    column_from_shifted = normed_ion_profile.iloc[:,11].dropna().to_numpy()
    display(column_from_shifted)
    assert np.allclose(column_from_shifted, column_from_shifted[0])

def test_that_profiles_with_noise_are_close():
    peptide1= lfq_testutils.PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=3000, add_noise=True)
    peptide2= lfq_testutils.PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=3, add_noise=True)
    peptide3= lfq_testutils.PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=0.1, add_noise=True)
    peptide4= lfq_testutils.PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)

    protein_df = lfq_testutils.ProteinProfileGenerator([peptide1, peptide2, peptide3, peptide4]).protein_profile_dataframe
    display(protein_df)
    
    normed_ion_profile = lfq_norm.normalize_ion_profiles(protein_df)
    display(normed_ion_profile)
    column_from_shifted = normed_ion_profile.iloc[:,9].dropna().to_numpy()

    assert np.allclose(column_from_shifted, column_from_shifted[0],rtol=0.01, atol=0.01)

test_that_profiles_without_noise_are_shifted_exactly_on_top_of_each_other()
test_that_profiles_with_noise_are_close()


0          1          2          3          4          5   \
protein ion                                                                     
protA   0    44.602965  44.474241  41.489085  43.216307  43.505708        NaN   
        1          NaN        NaN        NaN  33.250522        NaN        NaN   
        2    29.730290  29.601566  26.616410  28.343632  28.633033  27.465729   
        3          NaN        NaN        NaN        NaN        NaN        NaN   

                    6          7          8          9          10         11  \
protein ion                                                                     
protA   0    43.454211  43.102621  41.897534  43.625508  43.261340  40.203349   
        1          NaN        NaN        NaN        NaN        NaN        NaN   
        2    28.581537  28.229946  27.024859  28.752833  28.388665  25.330674   
        3          NaN        NaN  36.990643        NaN        NaN        NaN   

                    12  13         14        15         16         17  \
protein ion                                                             
protA   0    44.447364 NaN  44.692343  44.70169  43.106703  44.630975   
        1          NaN NaN  34.726559       NaN        NaN        NaN   
        2    29.574689 NaN  29.819668       NaN  28.234029  29.758301   
        3    39.540473 NaN        NaN       NaN        NaN        NaN   

                    18         19  
protein ion                        
protA   0    43.380650  43.407413  
        1          NaN        NaN  
        2    28.507975  28.534739  
        3          NaN        NaN

0          1          2          3          4          5   \
protein ion                                                                     
protA   0    44.602965  44.474241  41.489085  43.216307  43.505708        NaN   
        1          NaN        NaN        NaN  43.216307        NaN        NaN   
        2    44.602965  44.474241  41.489085  43.216307  43.505708  42.338404   
        3          NaN        NaN        NaN        NaN        NaN        NaN   

                    6          7          8          9         10         11  \
protein ion                                                                    
protA   0    43.454211  43.102621  41.897534  43.625508  43.26134  40.203349   
        1          NaN        NaN        NaN        NaN       NaN        NaN   
        2    43.454211  43.102621  41.897534  43.625508  43.26134  40.203349   
        3          NaN        NaN  41.897534        NaN       NaN        NaN   

                    12  13         14        15         16         17  \
protein ion                                                             
protA   0    44.447364 NaN  44.692343  44.70169  43.106703  44.630975   
        1          NaN NaN  44.692343       NaN        NaN        NaN   
        2    44.447364 NaN  44.692343       NaN  43.106703  44.630975   
        3    44.447364 NaN        NaN       NaN        NaN        NaN   

                   18         19  
protein ion                       
protA   0    43.38065  43.407413  
        1         NaN        NaN  
        2    43.38065  43.407413  
        3         NaN        NaN

array([40.20334853, 40.20334853])

0          1          2          3          4          5   \
protein ion                                                                     
protA   0    44.602965  44.474241  41.489086  43.216307  43.505708  42.338404   
        1    34.637186  34.508463  31.523287  33.250529  33.539917  32.372627   
        2    29.730236  29.601570  26.616213  28.343694  28.633051  27.465858   
        3    39.696073  39.567350  36.582195  38.309417  38.598820  37.431515   

                    6          7          8          9          10         11  \
protein ion                                                                     
protA   0    43.454211  43.102622  41.897535  43.625507  43.261340  40.203349   
        1    33.488411  33.136834  31.931726  33.659719  33.295545  30.237605   
        2    28.581627  28.230007  27.024935  28.752806  28.388671  25.330804   
        3    38.547322  38.195732  36.990636  38.718618  38.354450  35.296462   

                    12         13         14         15         16         17  \
protein ion                                                                     
protA   0    44.447364  44.554027  44.692343  44.701690  43.106703  44.630975   
        1    34.481560  34.588249  34.726569  34.735903  33.140917  34.665190   
        2    29.574561  29.681299  29.819709  29.828989  28.233933  29.758262   
        3    39.540472  39.647137  39.785451  39.794800  38.199811  39.724085   

                    18         19  
protein ion                        
protA   0    43.380650  43.407414  
        1    33.414869  33.441625  
        2    28.507947  28.534864  
        3    38.473758  38.500527

0          1          2          3          4          5   \
protein ion                                                                     
protA   0    44.602965  44.474241  41.489086  43.216307  43.505708  42.338404   
        1    44.602972  44.474249  41.489073  43.216315  43.505703  42.338413   
        2    44.602906  44.474240  41.488883  43.216364  43.505720  42.338528   
        3    44.602964  44.474240  41.489086  43.216307  43.505710  42.338405   

                    6          7          8          9          10         11  \
protein ion                                                                     
protA   0    43.454211  43.102622  41.897535  43.625507  43.261340  40.203349   
        1    43.454197  43.102620  41.897512  43.625505  43.261331  40.203391   
        2    43.454296  43.102677  41.897604  43.625475  43.261341  40.203474   
        3    43.454213  43.102622  41.897527  43.625508  43.261340  40.203352   

                    12         13         14         15         16         17  \
protein ion                                                                     
protA   0    44.447364  44.554027  44.692343  44.701690  43.106703  44.630975   
        1    44.447346  44.554035  44.692355  44.701690  43.106704  44.630976   
        2    44.447231  44.553969  44.692378  44.701658  43.106603  44.630931   
        3    44.447363  44.554027  44.692342  44.701690  43.106702  44.630975   

                    18         19  
protein ion                        
protA   0    43.380650  43.407414  
        1    43.380655  43.407411  
        2    43.380617  43.407534  
        3    43.380648  43.407417

In [5]:
import directlfq.protein_intensity_estimation as intensity_estimation
import directlfq.test_utils as lfq_testutils

def test_that_protein_intensities_are_retained():
    peptide1= lfq_testutils.PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=3000, add_noise=True)
    peptide2= lfq_testutils.PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=3, add_noise=True)
    peptide3= lfq_testutils.PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=0.1, add_noise=True)
    peptide4= lfq_testutils.PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    
    peptide_profiles = [peptide1, peptide2, peptide3, peptide4]
    summed_intensity_protein = sum([np.nansum(x.peptide_profile_vector) for x in peptide_profiles])
    
    protein_df = lfq_testutils.ProteinProfileGenerator([peptide1, peptide2, peptide3, peptide4]).protein_profile_dataframe
    protein_df_normed, _ = intensity_estimation.estimate_protein_intensities(protein_df, min_nonan=1, num_samples_quadratic=100, num_cores=1)
    display(protein_df_normed)
    display(protein_df_normed.iloc[0,1:].to_numpy())
    summed_lfq_intensities = np.sum(protein_df_normed.iloc[0,1:].to_numpy())
    assert np.allclose(summed_lfq_intensities, summed_intensity_protein)

test_that_protein_intensities_are_retained()



2024-01-16 18:41:04,627 - directlfq.protein_intensity_estimation - INFO - 1 lfq-groups total
2024-01-16 18:41:04,631 - directlfq.protein_intensity_estimation - INFO - lfq-object 0


,protein,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,protA,2.550681e+13,2.332953e+13,2.946332e+12,9.755077e+12,1.192199e+13,5.308341e+12,1.150395e+13,9.015887e+12,3.910578e+12,...,1.006437e+13,1.208460e+12,2.289902e+13,2.465628e+13,2.713699e+13,2.731334e+13,9.041394e+12,2.600682e+13,1.093207e+13,1.113679e+13


array([25506809860506.332, 23329533045893.477, 2946332363486.215,
       9755077268985.2, 11921993224575.64, 5308340747378.202,
       11503949328795.27, 9015887428968.75, 3910577876952.297,
       12954240952527.87, 10064371344868.074, 1208460069505.897,
       22899018922879.797, 24656284950204.83, 27136985048483.617,
       27313335493639.652, 9041394346226.639, 26006817726030.98,
       10932074976269.389, 11136794579192.637], dtype=object)

In [6]:
import directlfq.protein_intensity_estimation as intensity_estimation
import directlfq.test_utils as lfq_testutils

def run_with_multiple_proteins():
    peptide1= lfq_testutils.PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=3000, add_noise=True)
    peptide2= lfq_testutils.PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=3, add_noise=True)
    peptide3= lfq_testutils.PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=0.1, add_noise=True)
    peptide4= lfq_testutils.PeptideProfile(protein_name="protB",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide5= lfq_testutils.PeptideProfile(protein_name="protC",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide6= lfq_testutils.PeptideProfile(protein_name="protD",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide7= lfq_testutils.PeptideProfile(protein_name="protD",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide8= lfq_testutils.PeptideProfile(protein_name="protD",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)

    peptide_profiles = [peptide1, peptide2, peptide3, peptide4, peptide5, peptide6, peptide7, peptide8]
    protein_df = lfq_testutils.ProteinProfileGenerator(peptide_profiles).protein_profile_dataframe
    protein_df_normed, _ = intensity_estimation.estimate_protein_intensities(protein_df, min_nonan=1, num_samples_quadratic=100, num_cores=1)
    display(protein_df_normed)
    assert len(protein_df_normed.index) == 4

run_with_multiple_proteins()



2024-01-16 18:41:04,681 - directlfq.protein_intensity_estimation - INFO - 4 lfq-groups total
2024-01-16 18:41:04,683 - directlfq.protein_intensity_estimation - INFO - lfq-object 0


,protein,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,protA,2.377121e+13,2.174217e+13,2.745890e+12,9.091296e+12,1.111070e+13,4.947095e+12,1.072115e+13,8.402682e+12,3.644466e+12,...,9.379567e+12,1.126250e+12,2.134104e+13,2.297854e+13,2.529083e+13,2.545478e+13,8.426202e+12,2.423725e+13,1.018819e+13,1.037897e+13
1,protB,8.906542e+11,8.146273e+11,1.028809e+11,3.406309e+11,4.162956e+11,1.853568e+11,4.016985e+11,3.148188e+11,1.365504e+11,...,3.514312e+11,4.219766e+10,7.995915e+11,8.609485e+11,9.475760e+11,9.537380e+11,3.157105e+11,9.081165e+11,3.817288e+11,3.888777e+11
2,protC,8.906531e+11,8.146276e+11,1.028816e+11,3.406301e+11,4.162955e+11,1.853571e+11,4.016981e+11,3.148193e+11,1.365502e+11,...,3.514310e+11,4.219813e+10,7.995913e+11,8.609496e+11,9.475767e+11,9.537349e+11,3.157113e+11,9.081163e+11,3.817300e+11,3.888769e+11
3,protD,2.671962e+12,2.443883e+12,3.086453e+11,1.021890e+12,1.248887e+12,5.560694e+11,1.205094e+12,9.444571e+11,4.096502e+11,...,1.054291e+12,1.265943e+11,2.398776e+12,2.582841e+12,2.842731e+12,2.861211e+12,9.471338e+11,2.724346e+12,1.145189e+12,1.166631e+12


In [15]:
import directlfq.protein_intensity_estimation as lfq_intensity_estimation

def test_nameswitch_indices_are_recovered_correctly():
    # Example usage
    array = np.array(['a', 'a', 'a', 'b', 'c', 'c', 'd'])
    start_indices = lfq_intensity_estimation.find_nameswitch_indices(array)

    assert (array[start_indices[0]:start_indices[1]] == 'a').all()
    assert len(array[start_indices[0]:start_indices[1]]) == 3

    assert (array[start_indices[1]:start_indices[2]] == 'b').all()
    assert len(array[start_indices[1]:start_indices[2]]) == 1

    assert (array[start_indices[2]:start_indices[3]] == 'c').all()
    assert len(array[start_indices[2]:start_indices[3]]) == 2

    assert (array[start_indices[3]:start_indices[4]] == 'd').all()
    assert len(array[start_indices[3]:start_indices[4]]) == 1

test_nameswitch_indices_are_recovered_correctly()

## Learning tests

In [7]:
import pandas as pd
import numpy as np

def test_that_dataframe_is_generated_as_expected():
    vals1 = np.array([1, 2, 3,4 ])
    vals2 = np.array([5, 6, 7, 8])
    vals3 = np.array([9, 10, 11, 12])
    df = pd.DataFrame([vals1, vals2, vals3],index=['A', 'A', 'A'])
    display(df)
    assert df.iloc[2, 2] == 11
    assert df.iloc[1, 2] == 7




In [8]:

test_that_dataframe_is_generated_as_expected()

,0,1,2,3
A,1,2,3,4
A,5,6,7,8
A,9,10,11,12


In [9]:
def test_retrieval_of_numpy_arrays_from_dataframe():
    vals1 = np.array([1, 2, 3,4 ])
    vals2 = np.array([5, 6, 7, 8])
    vals3 = np.array([9, 10, 11, 12])
    df = pd.DataFrame([vals1, vals2, vals3],index=[['A', 'B', 'C'], ['a', 'b', 'a']])
    display(df)
    assert np.allclose(vals2, df.loc['B'])
    assert np.allclose([2, 6, 10], df.loc[:,1])


In [10]:

test_retrieval_of_numpy_arrays_from_dataframe()

,,0,1,2,3
A,a,1,2,3,4
B,b,5,6,7,8
C,a,9,10,11,12


In [11]:
def test_setting_numpy_seed():
    from numpy.random import MT19937
    from numpy.random import RandomState, SeedSequence

    rs = RandomState(MT19937(SeedSequence(42)))
    res = rs.randint(10,size=20)
    display(res)


In [12]:

test_setting_numpy_seed()

array([2, 6, 8, 8, 3, 3, 3, 3, 4, 7, 2, 7, 5, 4, 0, 8, 1, 3, 7, 1])